In [32]:
#set path and mean file
#%%capture
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image
import glob

caffe_root = './caffe/' 
import sys
sys.path.insert(0, caffe_root + 'python')
import caffe

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

mean_filename_female='./mean_image_female/mean.binaryproto'
mean_filename_male='./mean_image_male/mean.binaryproto'

proto_data_female = open(mean_filename_female, "rb").read()
proto_data_male = open(mean_filename_male, "rb").read()

a = caffe.io.caffe_pb2.BlobProto.FromString(proto_data_female)
mean_female  = caffe.io.blobproto_to_array(a)[0]

a = caffe.io.caffe_pb2.BlobProto.FromString(proto_data_male)
mean_male  = caffe.io.blobproto_to_array(a)[0]

caffe.set_mode_cpu()

In [36]:
#generate output from all 50 basic networks for a given set of images

labels = []
for ii in range(1):
    
    curIdx = ii+13
    age_net_pretrained_male='home/asus/Downloads/cnn2ranking/folds/train_val_txt_files_per_ranking/13_male.caffemodel'
    age_net_model_file='/home/asus/Downloads/cnn2ranking/folds/train_val_txt_files_per_ranking/deploy.prototxt'
    age_net = caffe.Classifier(age_net_model_file, age_net_pretrained_male,
                       mean=mean_male,
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

    labels.append([])
    #for filename in sorted(glob.glob('/home/asus/cnn2ranking/wiki_crop/01/595801_1967-09-09_2009.jpg')):
        
    #    im = caffe.io.load_image(filename)
     #   prediction = age_net.predict([im])
     #   label_tmp = prediction[0].argmax()
     #   labels[ind_lb].append(label_tmp)

RuntimeError: Could not open file ./home/asus/Downloads/cnn2ranking/folds/train_val_txt_files_per_ranking/13_male.caffemodel

In [ ]:
lb_all = np.array(labels) #binary labels
lb_sum = np.sum(lb_all, axis=0) #aggregated labels
lb_sum.shape

In [ ]:
#save results to txt files
np.savetxt('morph_te11k_results_bi.txt', lb_all, fmt='%d')

np.savetxt('morph_te11k_results_aggregate.txt', lb_sum, fmt='%d')

In [ ]:
#compare with ture labels and calculate MAE

load_in = np.genfromtxt('./link_to_data/tr43k_te11k_16to66/album2_faces_te11k_lb.txt', dtype=None)

lb_true = []
for i in range(len(load_in)):
    lb_true.append(load_in[i][1])

In [ ]:
lb_predict = lb_sum + 16

MAE_all = sum(abs(lb_true-lb_predict))*1.0/len(lb_true)
MAE_all